In [4]:
import pandas as pd
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader
from langchain_community.llms import Ollama

# credits to this code : https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/use-cases/document-summarization/summarization_large_documents_langchain.ipynb

In [ ]:
llm = Ollama(model="llama2")

# Load the PDF

In [2]:
pdf_loader = PyPDFLoader("../pdf_data/allcott.pdf")
pages = pdf_loader.load_and_split()
print(pages[3].page_content)

Our main GPS results show that the strong partisan differences in social distancing behavior
that emerged with the rise of COVID-19 are not merely an artifact of differences in public policies
or observed risks. Controlling for state-time ﬁxed effects to account for heterogenous policy re-
sponses by state governments only attenuates the partisan gap slightly. Including controls to proxy
for local policy, health, weather, and economic variables interacted ﬂexibly with time attenuates
the gap more substantially, but it remains statistically and economically signiﬁcant. After including
our full set of controls, we estimate that moving from the 10th to the 90th percentile of Republican
county vote share is associated with 11.5 and 15.2 percent increases in the number of POI visits
during the weeks of April 6 and May 11, when social distancing and partisan gaps are at their
respective peaks.
Our ﬁndings are robust to the inclusion or exclusion of control variables, excluding states with
ea

# First method : Stuffing (i.e. considering the PDF in a single call)

In [3]:
prompt_template = """Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
  """

prompt = PromptTemplate(template=prompt_template, input_variables=["text"])

In [6]:
stuff_chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)

In [7]:
try:
    print(stuff_chain.run(pages))
except Exception as e:
    print(
        "The code failed since it won't be able to run inference on such a huge context and throws this exception: ",
        e,
    )

c:\Users\pvanb\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


1. The appendix includes seven figures that illustrate various aspects of the study's findings, including the effect of partisan affiliation on beliefs and actions related to COVID-19, the impact of incentives on beliefs and actions, and the relationship between news consumption and partisan affiliation.
2. The figures show that:
	* Beliefs and actions related to COVID-19 are influenced by partisanship, with Republicans being less concerned about COVID-19 than Democrats.
	* Incentives can affect beliefs and actions related to COVID-19, with incentivized subjects more likely to report engaging in social distancing behaviors than unincentivized subjects.
	* News consumption is associated with partisan affiliation, with Republicans being more likely to consume news from sources that are favorable to their party than Democrats.
3. The study used a survey of over 1,000 Americans to collect data on beliefs and actions related to COVID-19, as well as demographic information and news consumpti

# Second method : Map-reduce (small chunks that are passed to the LLM one by one)

In [8]:
map_prompt_template = """
                      Write a summary of this chunk of text that includes the main points and any important details.
                      {text}
                      """

map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])

combine_prompt_template = """
                      Write a concise summary of the following text delimited by triple backquotes.
                      Return your response in bullet points which covers the key points of the text.
                      ```{text}```
                      BULLET POINT SUMMARY:
                      """

combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=["text"]
)

In [9]:
map_reduce_chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=combine_prompt,
    return_intermediate_steps=True,
)

In [10]:
map_reduce_outputs = map_reduce_chain({"input_documents": pages})

c:\Users\pvanb\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
